<a href="https://colab.research.google.com/github/hsym-uec/myWeb_github/blob/main/Simmple_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# SoftBank Stock data from Stooq.com 
# reference https://seanmemo.com/234/

from pandas_datareader import data
start = '2023-05-10'
end = '2023-05-17'
df = data.DataReader('9984.JP', 'stooq', start, end)


In [19]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2023-05-16,5009,5047,4986,4999,8285300
2023-05-15,4940,5014,4879,4950,9989400
2023-05-12,4984,5010,4854,4949,22738400
2023-05-11,5151,5171,5102,5138,6409200
2023-05-10,5120,5189,5098,5182,7001600


In [22]:
df.iat[0, 3]

4999

In [26]:
import datetime
from datetime import date, timedelta
dt_yesterday = datetime.datetime.now() - timedelta(days=1)
dt_yesterday = dt_yesterday.strftime('%Y-%m-%d')
print(dt_yesterday)

2023-05-16


In [66]:
### update balance

import mysql.connector
import datetime

def insert_balance(balance):

    cnx = None
    dt_now = datetime.datetime.now()

    try:
        cnx = mysql.connector.connect(
            user='root',  # ユーザー名
            password='',  # パスワード
            host='localhost',  # ホスト名(IPアドレス）
            database='stock_test'  # データベース名
        )

        cursor = cnx.cursor()

        sql = ('''
        INSERT INTO wallent02 
            (balance, datetime)
        VALUES 
            (%s, %s)
        ''')

        data = [
            (balance, dt_now)
        ]

        cursor.executemany(sql, data)
        cnx.commit()

        print(f"{cursor.rowcount} records inserted.")

        cursor.close()

    except Exception as e:
        print(f"Error Occurred: {e}")

    finally:
        if cnx is not None and cnx.is_connected():
            cnx.close()


insert_balance(1000000)


1 records inserted.


In [74]:
### get latest balance

import mysql.connector

def get_total_balance():

    cnx = None
    current_balance = 0

    try:
        cnx = mysql.connector.connect(
            user='root',  # ユーザー名
            password='',  # パスワード
            host='localhost',  # ホスト名(IPアドレス）
            database='stock_test'  # データベース名
        )

        cursor = cnx.cursor()

        sql = ('''
        SELECT balance
        FROM wallent02
        ''')

        cursor.execute(sql)
        #cnx.commit()

        print(f"Last balance gotten.")

        for row in cursor.fetchall():
          current_balance += row[0]

        cursor.close()

    except Exception as e:
        print(f"Error Occurred: {e}")

    finally:
        if cnx is not None and cnx.is_connected():
            cnx.close()
        return int(current_balance)


get_total_balance()

Last balance gotten.


905450

In [72]:
# get latest closed value

from pandas_datareader import data
import datetime
from datetime import date, timedelta

def get_closed(company_num):
    dt_yesterday = datetime.datetime.now() - timedelta(days=1)
    dt_yesterday = dt_yesterday.strftime('%Y-%m-%d')

    start = dt_yesterday
    end = dt_yesterday

    company_code = str(company_num) + '.JP'
    df = data.DataReader(company_code, 'stooq', start, end)

    return df.iat[0, 3]  # 昨日の終値


get_closed(9984)


4999

In [75]:
# purchase the stocks
def purchase_stocks(company_num, purchase_num):
    stock_closed = get_closed(company_num).item()
    purchase_amount = stock_closed * purchase_num  # 購入金額 = 昨日の終値 * 購入数

    insert_balance((-1) * purchase_amount)  # 手持ち金額を精算

    cnx = None
    dt_now = datetime.datetime.now()

    try:
        cnx = mysql.connector.connect(
            user='root',  # ユーザー名
            password='',  # パスワード
            host='localhost',  # ホスト名(IPアドレス）
            database='stock_test'  # データベース名
        )

        cursor = cnx.cursor()

        sql = ('''
        INSERT INTO stock_holdings06 
            (company_code, amount, purchased_rate, yesterday_rate, updated)
        VALUES 
            (%s, %s, %s, %s, %s)
        ''')

        data = [
            (company_num, purchase_num, stock_closed, stock_closed, dt_now)
        ]

        cursor.executemany(sql, data)
        cnx.commit()

        print(f"{cursor.rowcount} records inserted.")

        cursor.close()

    except Exception as e:
        print(f"Error Occurred: {e}")

    finally:
        if cnx is not None and cnx.is_connected():
            cnx.close()
    

purchase_stocks(6752, 10)

1 records inserted.
1 records inserted.
